# Using Tensorboard to visualize model structure, loss , and accuracy

| model structure | loss and accuracy |
| --- | --- |
| ![test](https://imgur.com/ijOoUsx.png) | ![test](https://imgur.com/8PsKdMQ.png) |

## Building CNN model on MNIST dataset

## What is MNIST dataset?

### MNIST is a simple computer vision dataset. 
### It consists of images of handwritten digits like these:



![test](https://imgur.com/9w9uz2r.png)

### Each image in MNIST has a corresponding label, a number between 0 and 9 representing the digit drawn in the image, for the pictures above the labels are: "5 , 0 , 4 , 1"

# Let's begin our task (๑•̀ㅂ•́)و✧

# Import

In [1]:
#Import tensorflow library and mnist dataset
from __future__ import print_function
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)
/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Create model

## Tensorboard : with tf.name_scope('name'): 

![test](https://imgur.com/hlldMpv.png)

## Create placeholder (tutorial PPT page 7)

In [2]:
# define placeholder for inputs to network
with tf.name_scope('Inputs'):
    xs = tf.placeholder(tf.float32, [None, 784]) # 28x28
    x_image = tf.reshape(xs, [-1, 28, 28, 1])
with tf.name_scope('Labels'):
    ys = tf.placeholder(tf.float32, [None, 10])
with tf.name_scope('Placeholder_for_Dropout'):
    keep_prob = tf.placeholder(tf.float32)


## Add layers (tutorial PPT page 9~23)

![test](https://imgur.com/KZ8BnJa.png)

### First, you should have the concept of model structure mentioned in page 22)

In [3]:
with tf.name_scope('model'):
    
    ## conv1 layer ##
    with tf.name_scope('conv1'):
        # page 8 + the exercises minutes ago
        w_conv1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1)) #filter 5x5, in channel size 1, out size 32
        b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
        # page 13 + the exercises minutes ago
        conv1 = tf.nn.conv2d(x_image, w_conv1, strides=[1,1,1,1], padding='SAME')
        
    #activation function
    with tf.name_scope('relu'):
        # page 20 + the exercises minutes ago
        activate_conv1 = tf.nn.relu(conv1 + b_conv1) #output size 28x28x32

    ## pool1 layer ##
    with tf.name_scope('pool1'):
        # page 13 + the exercises minutes ago
        max_pool1 = tf.nn.max_pool(activate_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') #output size 14x14x32 


    ###############You can add your own layers if you want################
    # Make sure the channel in should be 32 which is mentioned above or you can calculate by yourself :-)
    # Here is an example of another layers:conv2+pool2
    '''
    ## conv2 layer ##
    with tf.name_scope('conv2'):
        w_conv2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
        b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
        conv2 = tf.nn.conv2d(max_pool1, w_conv2, strides=[1,1,1,1], padding='SAME')
        
    #activation function
    with tf.name_scope('relu'):
        activate_conv2 = tf.nn.relu(conv2 + b_conv2)

    ## pool2 layer ##
    with tf.name_scope('pool2'):
        max_pool2 = tf.nn.max_pool(activate_conv2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
     
    '''
    ######################################################################
    
    # page 22-23 + the exercises minutes ago 

    with tf.name_scope('flatten'):
        '''If you've added layers above make sure that your flatten layer has attached to the last layer.
        For the example given above, the code below should be like:
        h_flat = tf.contrib.layers.flatten(max_pool2)'''
        h_flat = tf.contrib.layers.flatten(max_pool1)

    ## fc1 layer ##
    with tf.name_scope('fc1'):
        w_nn1 = tf.Variable(tf.truncated_normal([h_flat.shape[1].value, 1024], stddev = 0.1))
        b_nn1 = tf.Variable(tf.constant(0.1, shape=[1024]))
    
    #activation function
    with tf.name_scope('relu'):
        activate_nn1 = tf.nn.relu(tf.matmul(h_flat, w_nn1) + b_nn1)
        
    #dropout
    with tf.name_scope('dropout'):
            nn1_drop = tf.nn.dropout(activate_nn1, keep_prob)
    
    ## fc2 layer ##
    with tf.name_scope('fc2'):
        w_nn2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
        b_nn2 = tf.Variable(tf.constant(0.1, shape=[10]))
    
    ## output layer ##
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(tf.matmul(nn1_drop, w_nn2) + b_nn2)
        

# Compute loss function (tutorial PPT page 34)

In [4]:
with tf.name_scope('Loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction), reduction_indices=[1]))
    tf.summary.scalar("Loss", cross_entropy) # for tensorboard

# Optimization (tutorial PPT page 36) 

##  You can modify the learning rate: 1e-4

## But make sure not to set the learning rate too low , or your VM would be very very slow 

In [5]:
with tf.name_scope('Optimization'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Computing accuracy

## For example, if the number of test samples is 1000 and model classifies 952 of those correctly, then the model's accuracy is 95.2%.

In [6]:
with tf.name_scope('Accuracy'):
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(ys, 1))
    acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar('Accuracy', acc) # for tensorboard

# Start training and recording in Tensorboard

## For training part, you can find more descriptions from the official website

### Chinese:  http://www.tensorfly.cn/tfdoc/tutorials/mnist_pros.html

### English:  https://www.tensorflow.org/versions/r1.2/get_started/mnist/beginners

In [ ]:
################################ This section may take a few minutese. Please be patient. ################################ 

logs_path = 'TensorBoard/' # save the tensorboard file in this folder
merged = tf.summary.merge_all() # for tensorboard
writer = tf.summary.FileWriter(logs_path , graph = tf.get_default_graph()) # for tensorboard

sess = tf.Session()
sess.run(tf.initialize_all_variables())

# The following code means we train 600 times and each time we feed in 10 pictures
for i in range(600): # We call it 600 epochs
    batch_xs, batch_ys = mnist.train.next_batch(10)
    sess.run(train_step, feed_dict = {xs: batch_xs, ys: batch_ys, keep_prob:0.5} )
    if i % 100 == 0:
        summary = sess.run(merged, feed_dict = {xs: batch_xs, ys: batch_ys, keep_prob:0.5})
        writer.add_summary(summary, i) # for tensorboard
        
writer.close() # for tensorboard      
sess.close() # for tensorboard

print("\n(*´▽`*)!!!finish!!!(*´▽`*)")

Instructions for updating:
Use `tf.global_variables_initializer` instead.


#  Open tensorboard

## $ tensorboard --logdir='~/summer-school-2018-tw/06-ai-deep-learning/TensorBoard/'

## Type <span style="color:blue">localhost:6006</span> at your web browser

![test](https://imgur.com/IkvXqnn.png)

## You may wonder what is the difference between epoch and step in tensorboard

### You can look for more  descriptions in the link below:

### https://stackoverflow.com/questions/38340311/what-is-the-difference-between-steps-and-epochs-in-tensorflow

## After training, make sure to clean up the earlier records in the TensorBoard folder and restart the terminal.Otherwise your board will look like a mess.